In [1]:
import pandas as pd
import os
import numpy as np
import wfdb
import ast
import paramiko
import matplotlib.pyplot as plt
import seaborn as sns
# import cv2


In [2]:
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data


path = ''
ptbxl_database = 'ptbxl_database.csv'
ptbxl_records = 'records100//00000//00001_lr.hea'
save_path = '/Visual results'
sampling_rate=100

# load and convert annotation data
Y = pd.read_csv(path + ptbxl_database, index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load raw signal data
X = load_raw_data(Y, sampling_rate, path)

# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

In [3]:
# Apply diagnostic superclass
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

In [4]:
#Split data into train and test
test_fold = 10
# Train
X_train = X[np.where(Y.strat_fold != test_fold)]
y_train = Y[(Y.strat_fold != test_fold)].diagnostic_superclass
# Test
X_test = X[np.where(Y.strat_fold == test_fold)]
y_test = Y[Y.strat_fold == test_fold].diagnostic_superclass

In [7]:
Y.strat_fold.value_counts() #To strattify 10 groups for training and 2 for testing
base_name = "ECG_plot"

for w in range(22000):
    sample = X[w]
    bar, axes = plt.subplots(sample.shape[1], 1, figsize=(20,10))
    for i in range(sample.shape[1]):
        sns.lineplot(x=np.arange(sample.shape[0]), y=sample[:, i], ax=axes[i])
        my_path = os.path.abspath('Visual results')
        my_file = f"{base_name}{w}.png"
        plt.savefig(my_path+"/"+my_file)
        #plt.savefig(os.path.join(my_path))
    #plt.show()
       # plt.close()

NameError: name 'Y' is not defined